In [32]:
import aiohttp, http, asyncio, requests, re, time, os, json, pickle
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import steam_tags_f as stf

In [33]:
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 16)

In [ ]:
%load_ext autoreload
%autoreload 2 

GETTING THE TAGS AND MULTIPLAYER INFO

In [ ]:
url = "http://api.steampowered.com/ISteamApps/GetAppList/v2/"
token = "1AC3575ADB761319F3EF72D1FA362232"
params = {"key":token}

scode, data = get_url(url, params=params, type = "api")
display(scode)

In [35]:
app_dict = {app['name']: app['appid'] for app in data['applist']['apps']}

In [10]:
top1000 = pd.read_csv('TOP 1000 Games cleaned.csv')

In [36]:
top1000["APP ID"] = 0

In [ ]:
#top1000_2 = stf.add_appid(top1000,app_dict)

In [ ]:
#top1000.to_csv('TOP 1000 Games cleaned AppId.csv', index=False)

In [51]:
#manually entered some app-ids that were not fetched
appid_fix = pd.read_csv('top_1000_games_cleaned_appid_fix.csv')

In [ ]:
appid_fix

In [53]:
txt = pd.read_csv(r"top_1000_games_cleaned_appid_fix.csv")
txt = txt.rename({"Game Title":"title", "Steam App ID":"id"}, axis=1)
appid_list = list(txt["id"])

In [ ]:
# Example
appid = 2923300

# Step 1: Make a request to the Steam API
response = requests.get(f"http://store.steampowered.com/api/appdetails?appids={appid}")

# Step 2: Parse the response
game = response.json()

#Step 3: Pretty-print the JSON response for better readability
print(json.dumps(game, indent=4))  # Format with indentation of 4 spaces


In [ ]:
genres = game["2923300"]["data"]["genres"]
descriptions = [genre["description"] for genre in genres]
print (descriptions)

In [ ]:
result = stf.check_multiplayer_single("1599340")
print(result) 

In [ ]:
#tag_dict = stf.get_tags(appid_list)

In [57]:
with open ("tag_dict.pkl", "rb") as tag_dict_file:
    tag_dict = pickle.load(tag_dict_file)


In [ ]:
tag_dict

In [59]:
appid_fix["tags"] = 0

#steam_df_tags = stf.add_tags(appid_fix,tag_dict)

In [ ]:
with open ("steam_df_tags.pkl", "rb") as steam_df_tags_file:
    steam_df_tags = pickle.load(steam_df_tags_file)

steam_df_tags

In [ ]:
#steam_df_tags.to_csv('steam_appid_tags.csv', index=False)

In [61]:
token = "1AC3575ADB761319F3EF72D1FA362232"

In [ ]:
#mp_dict = stf.check_multiplayer(appid_list, token, delay=1)

In [ ]:
with open ("mp_dict.pkl", "rb") as mp_dict_file:
    mp_dict = pickle.load(mp_dict_file)

mp_dict

In [63]:
steam_df_tags_mp = stf.add_mp(steam_df_tags, mp_dict)

In [ ]:
steam_df_tags_mp

In [65]:
steam_df_tags_mp.to_csv('steam_appid_tags_mp.csv', index=False)

In [66]:
tags_mp_dict = steam_df_tags_mp.set_index("Steam App ID")["tags"].to_dict()

In [ ]:
tags_mp_dict

In [ ]:
#with open ("tags_mp_dict.pkl", "wb") as tags_mp_dict_file:
    #pickle.dump(tags_mp_dict, tags_mp_dict_file)

In [68]:
tags_name_dict = steam_df_tags_mp.set_index("Steam App ID")["Game Title"].to_dict()

In [ ]:
tags_name_dict[226320]

In [ ]:
tags_name_dict

In [ ]:
#with open ("tags_name_dict.pkl", "wb") as tags_name_dict_file:
   # pickle.dump(tags_name_dict, tags_name_dict_file)

In [ ]:
out3_extracted.head(10)

In [ ]:
#tags_mp_dict

In [ ]:
#for transposing new_df = out3_extracted.T.copy()

new_df = out3_extracted.copy()

In [ ]:
new_df.columns

In [ ]:
new_row = pd.DataFrame(tags_mp_dict, index = [0])

In [ ]:
add row [0]
for col in new_df.columns:
    if col in tags_mp_dict.keys()
        row [0][column] = tags_mp_dict[key]

In [ ]:
# Define a new row of zeroes with the same number of columns as df
zero_row = pd.DataFrame([[0] * len(new_df.columns)], columns=new_df.columns)

# Concatenate the new zero row at index 0
new_df_0 = pd.concat([zero_row, new_df]).reset_index(drop=True)

In [ ]:
new_df_0

In [ ]:
tags_mp_dict

In [ ]:
tags_mp_dict_str = {str(key):value for key, value in tags_mp_dict.items()}

In [ ]:
tags_mp_dict_str

In [ ]:
df2 = add_tags_df(new_df_0, tags_mp_dict_str)
df2.head()

In [ ]:
tags_name_dict

In [ ]:
"""
def add_tags_df (df, dict):
    df2 = df.copy()
    for col in df2.columns:
        int_key = int(col)
        if int_key in map (int, dict.keys()):
            df2.at[0, col] = dict[str(col)]
    return df2
"""        
        

In [ ]:
"""
for col in df2.columns:
        # Convert the integer key to string for comparison
        str_key = str(col)
        if str_key in map(str, tags_mp_dict.keys()):  # Check if the string version of the key exists in dictionary
            df2.at[0, col] = tags_mp_dict[int(col)]  # Use .at to set value by label

    return df2
"""

In [ ]:
df2

FUNCTIONS

In [75]:
def get_url(url, headers=None, params=None, type="ws"):
    """
    Args:
        url (str): pass on url
        type (str): define "ws" (webscrapping) or "api" (for API/json output), default is ws
        headers (str, optional): Defaults to None.
        params (dict, optional): Defaults to None.

    Returns:
        stat , response: outputs stat code and response content, either soup or json type.
    """
    response = requests.get(url,headers=headers, params=params)
    stat = response.status_code
    if type.lower() == "ws":
        soup = BeautifulSoup(response.content, 'html.parser')
    elif type.lower() == "api":
        soup = response.json()
    time.sleep(1)
    
    return stat, soup

def fetch_dict(soup, gameid):
    k_value = 0
    players = soup.find("tbody")
    play = re.split(r"[\n\t]+",players.get_text().strip())
    player_game = {} 

    for i in range(0, len(play), 5):
        # Date
        date = play[i]
        # Peak players
        peak = play[i+1]  
        # Average players
        average = play[i +4]
        player_game[k_value]={"Date": date,
                            "Peak Players":peak,
                            "Avg Players":average,
        }
        
        k_value += 1
        
    return player_game

def overall_fetch(list_id):
    error_list =[]
    game_id_dict = {}
    for gameid in list_id:
        gameid = str(gameid)
        url = f'https://steamcharts.com/app/{gameid}'
        headers= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}
        try:
            stat, soup = get_url(url, headers=headers, params=gameid)
            if stat == 200:
                print(f"Webcode status is : {stat} for {gameid} id. ")
                player_game = fetch_dict(soup, gameid)
                game_id_dict[gameid] = player_game.copy()
            else:
                error_list.append((gameid, stat))
                print(f"Appid: {gameid} reported an error {stat}, skipping it for now")    
        except Exception as e:  # Catch specific exceptions
            error_list.append((gameid, stat))
            print(f"App ID: {gameid} reported an error: {str(e)}, skipping it for now.")
            
    
    return game_id_dict, error_list

def add_tags_df (df, dict):
    df2 = df.copy()
    for col in df2.columns:
        if col in dict.keys():
            df2.at[0, col] = dict[col]
    return df2

MAIN CODES

In [12]:
# creates the games id list
txt = pd.read_csv(r"C:\\Users\\Utilizador\\Desktop\\IRONHACK\\Project 3\\Project-3\\top1k.csv")
txt = txt.rename({"Game Title":"title", "Steam App ID":"id"}, axis=1)
list_id = list(txt["id"])

In [ ]:
## main program, output

game_id_dict, error_list = overall_fetch(list_id)

In [ ]:
# Saves our main source of information into a feather
output = pd.DataFrame.from_dict(game_id_dict)
output.to_feather("output.feather")

In [ ]:
# Saves our error results into an error csv
err_table = pd.DataFrame(error_list)
err_table.to_csv("error.csv")

In [ ]:
### Bridges the webscrapping/API part with the cleaning and processing parts

# imports our feather database & gives us an idea of the shape/NaN's present
output = pd.read_feather("C:\\Users\\Utilizador\\Desktop\\IRONHACK\\Project 3\\Project-3\\output.feather")
output.isna().sum()
display(output.shape)

In [72]:
# Cleans database of most recent games (so from Aug 2019 -> Present)
out2 = output.copy() 
filtered = out2.isna().sum(axis=0) > 85 ## 85 = Aug 2019
filter = filtered[filtered]

mask = list(filter.index)
for code in mask:
    out2.drop(code, axis = 1, inplace=True)

In [ ]:
# Converts "Date" into DataFrame index
out3=out2.copy()
out2_extracted = out2.applymap(lambda x: x["Date"] if isinstance(x, dict) else x) 

out3["Date"] = out2_extracted["730"]

# Define a new row of zeroes with the same number of columns as df
zero_row = pd.DataFrame([[0] * len(out3.columns)], columns=out3.columns)

# Concatenate the new zero row at index 0
new_df_0 = pd.concat([zero_row, out3]).reset_index(drop=True)
tags_mp_dict_str = {str(key):value for key, value in tags_mp_dict.items()}

# Adds tag into row 0, also gives the column a name
out3_new = add_tags_df(new_df_0, tags_mp_dict_str)
out3_new["Date"][0] = "Tags"

# Set's date as index
out3_new.set_index('Date', inplace=True)

# filters the table for the data we want
out3_extracted = out3_new.applymap(lambda x: x["Avg Players"] if isinstance(x, dict) else x)

display(out3_extracted)

In [ ]:
# Converts columns into game's name
out4 = out3_extracted.copy()
# creates the id list
txt2 = txt.rename({"Game Title":"title", "Steam App ID":"id"}, axis=1)
txt2 = txt2[["id","title"]]
txt2.set_index("id", inplace=True)

# Create a mapping from ids to titles
title_mapping = {str(k): v for k, v in txt2["title"].to_dict().items()}

# Rename the columns in out4 using the mapping
out4.columns = [title_mapping.get(str(col), col) for col in out4.columns]

# Creates our main dataframe visualization 
visual = "Peak Players"
out4 = out4.applymap(lambda x: x[visual] if isinstance(x, dict) else x)
out4[1:] = round(out4[1:].fillna(0).astype(float),0).astype(int)
out4

In [ ]:
# More cleaning while looking at data. Disregarding outliers etc..
out5 = out4.copy()

## Drops useless games that make weird spikes
out5.drop("POSTAL", axis=1, inplace=True)

# Creates a new column (Total) and concats into the existing dataframe
total_column = out5.sum(axis = 1)
out5 = pd.concat([out5, total_column.rename("Total")], axis=1)

out5

In [ ]:
# Normalization 1 (average gains) - Assuming 'Total' is the column you want to apply the formula to
#out5['Value'][1:] = (out5['Total'][1:] - out5['Total'][1:].shift(-1)) / out5['Total'][1:]

out5["Value"] = out5["Total"]
out5["Value"] = 0
out5['Value'][1:] = (out5['Total'][1:] - out5['Total'][1:].shift(-1)) / out5['Total'][1:]

display(out5)
#display(txt2)

In [ ]:
# new copy
out6 = out5.copy()

# further normalizations - Z score normalization
def z_norm(column):
    mean = column.mean()
    std_dev = column.std()
    return (column - mean) / std_dev

out6['Z'] = out6['Value']
out6['Z'] = 0
out6['Z'][1:] = z_norm(out6['Total'][1:])

out6

In [ ]:
###
out7 = out6.copy()

print(out7.index)

In [ ]:


filter_tags = out7[out7.index.str.contains('Multiplayer', case=False, na=False)]
print(filter_tags)

In [ ]:


## Data Visualization 

# Set theme and palette
sns.set_theme(style="darkgrid")
sns.set_palette("viridis")

# Create a line plot for Total values over Date
plt.figure(figsize=(12, 6))  # Adjust figure size
sns.lineplot(data=out7[1:], x=out7.index[1:], y='Z', marker='o', linestyle='--', linewidth=2)

#### Add a vertical line on a specific date

# Covid related lines
plt.axvline(x="November 2019", color='orange', linestyle='--', linewidth=1)
plt.axvline(x="January 2020", color='red', linestyle='--', linewidth=1)
plt.axvline(x="June 2020", color='green', linestyle='--', linewidth=1)

plt.text(x="November 2019", y=-2,  s='Nov19: China warns about Covid', color='orange', fontsize=10, ha='right', rotation=90)
plt.text(x="January 2020", y=-2, s='Jan20: WHO acknoledges Covid', color='red', fontsize=10, ha='left', rotation=90)
plt.text(x="June 2020", y=-2, s='Jun20: end of first big lockdown period', color='green', fontsize=10, ha='right', rotation=90)

# Game related lines
plt.axvline(x="December 2017", color='cyan', linestyle='--', linewidth=1)
plt.axvline(x="December 2020", color='cyan', linestyle='--', linewidth=1)
plt.axvline(x="September 2021", color='cyan', linestyle='--', linewidth=1)
plt.axvline(x="February 2021", color='cyan', linestyle='--', linewidth=1)
plt.axvline(x="March 2019", color='cyan', linestyle='--', linewidth=1)


plt.text(x="December 2017", y=-2, s="PUBG goes live", color='cyan', fontsize=10, ha='right', rotation=90)
plt.text(x="December 2020", y=-2, s="Cyberpunk goes live", color='cyan', fontsize=10, ha='right', rotation=90)
plt.text(x="September 2021", y=-2, s="New World goes live", color='cyan', fontsize=10, ha='right', rotation=90)
plt.text(x="February 2021", y=-2, s="Valheim goes live", color='cyan', fontsize=10, ha='right', rotation=90)
plt.text(x="March 2019", y=-2, s="Sekiro goes live", color='cyan', fontsize=10, ha='right', rotation=90)

### Graphic formatting

# Format the date on the x-axis
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))

#Limit x-axis onto a specific interval 
plt.xlim('August 2022', 'August 2018')  # Specify the limits as strings
plt.gca().invert_xaxis() # inverts the x-axis (more suited for dates as it grows from left to right)

# Set titles and labels
plt.title('Z-values Over Time')
plt.xlabel('Date')
plt.ylabel('Z-Value')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability